In [6]:
import os
import glob
import json
import faiss
import numpy as np
from langchain.text_splitter import CharacterTextSplitter
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize


# 1. KoSBERT 모델 로드
model = SentenceTransformer('jhgan/ko-sbert-sts')

# 
# 2. JSON 파일 읽기
folder_path = "./new2"
json_files = glob.glob(os.path.join(folder_path, "*.json"))

raw_documents = []
for file in json_files:
    with open(file, "r", encoding="utf-8-sig") as f:
        data = json.load(f)
        # 최상위에 "data" 키가 있고, 그 안에 리스트 구조가 있다고 가정
        if "data" in data and isinstance(data["data"], list):
            for item in data["data"]:
                # corpus 필드가 있는 경우에만 추가
                if "corpus" in item:
                    raw_documents.append(item["corpus"])


print(f"✅ 총 {len(raw_documents)}개의 JSON 문서를 로드했습니다.")

# =========================
# 3. FAISS 인덱스 생성 함수
# =========================
def create_faiss_index(texts, model):
    if not texts:
        raise ValueError("❌")
    embeddings = model.encode(texts, convert_to_numpy=True)
    if embeddings.ndim == 1:
        embeddings = embeddings.reshape(1, -1)
    embeddings = normalize(embeddings, axis=1)
    index = faiss.IndexFlatIP(embeddings.shape[1])
    index.add(embeddings)
    return index, embeddings

# =========================
# 4. 유사 문서 검색 함수
# =========================
def retrieve_documents(query, index, model, texts, top_k=3):
    query_embedding = model.encode([query], convert_to_numpy=True)
    query_embedding = normalize(query_embedding, axis=1)
    distances, indices = index.search(query_embedding, top_k)
    return [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]

# =========================
# 5. 청킹 및 검색 실행 함수
# =========================
def process_chunking_and_retrieval(chunk_size, chunk_overlap, texts, query, model):
    text_splitter = CharacterTextSplitter(
        separator="",  # 줄바꿈 단위가 아닌 문자 단위 청크
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    joined_text = "\n".join(texts)
    chunked_texts = text_splitter.split_text(joined_text)

    if not chunked_texts:
        raise ValueError("❌ 청킹된 텍스트가 없습니다.")

    index, _ = create_faiss_index(chunked_texts, model)
    retrieved = retrieve_documents(query, index, model, chunked_texts, top_k=3)
    return retrieved

# =========================
# 6. 질의 및 청킹 세트 정의
# =========================
query = "횡성군 종합민원실의 민원행정서비스 조사는 며칠을 선정하여 실시하였어?"
chunking_configs = [
    (500, 50)
]

# =========================
# 7. 실행
# =========================
for chunk_size, chunk_overlap in chunking_configs:
    print(f"\n===============================")
    print(f"🔍 청킹 설정: {chunk_size}+{chunk_overlap}")
    print(f"===============================")
    try:
        results = process_chunking_and_retrieval(chunk_size, chunk_overlap, raw_documents, query, model)
        for i, (doc, score) in enumerate(results):
            print(f"\n▶️ [Top {i+1}] 유사도: {score:.4f}")
            print(doc[:700], "...\n")
    except Exception as e:
        print(f"⚠️ 오류 발생: {e}")



✅ 총 10개의 JSON 문서를 로드했습니다.

🔍 청킹 설정: 500+50

▶️ [Top 1] 유사도: 0.6761
로 장기발전전략 수립
▷ 고객만족도 제고를 위한 기초조사실시의 필요성 부각
▷ 눈높이 민원행정의 실현
2. 연구의 목적
횡성군 종합민원실에서 제공하는 행정서비스의 고객인 이용자들로 하여금 민원행정서비스의 전달체계와 그 서비스 과정에서 공무원들의 행태와 민원 제도, 시설 등에 대하여 체감정도를 조사를 목적으로 하고 있다. 
횡성군 민원행정 서비스 고객만족도 조사는 구체적으로 다음과 같은 의의를 내포하고 있다. 
▷ 고객중심의 초일류 민원행정서비스의 실현
▷ 고객만족도 제고 및 고객감동 지향
▷ 불합리한 제도개선과 발전방향 제시
▷ 맞춤형 민원행정 서비스 실현
▷ 전자민원 행정구현을 위한 기초조사
▷ 행정업무 분야별 만족도 제고
▷ 행정서비스의 취약점과 개선방향을 도출
▷ 과거의 고객만족도 결과와 비교하여 고객만족도 수준의 변화정도를 분석고객만족도 조사는 궁극적으로 평가대상이 되는 종합민원실로 하여금 고객중심의 행정서비스를 구현하며, 행정서비스의 질을 향상시킴으로써 횡성군의 경쟁력 제고 ...


▶️ [Top 2] 유사도: 0.6384
제1장 연구개요
I. 연구배경 및 목적
1. 연구의 배경민원행정서비스에 대한 만족도 조사는 한국행정연구원에서 1996년 조사모델과 방법을 개발한 이후 현재까지 지속적으로 수행하고 있는 중앙정부 및 지방정부의 과제이다. 
횡성군 민원행정서비스 만족도조사는 횡성군 종합민원실에서 제공하는 민원행정서비스를 이용한 지역주민과 이용객을 대상으로 하여 행정서비스의 전달체계와 그 서비스 과정에서의 공무원들의 행태와 민원제도 등에 대해 체감하는 만족도를 조사하고자 실시되었다. 
특히 상반기조사결과를 비교 대상으로 하여 하반기 고객 만족도의 수준을 평가하고 발전지향적인 고객중심의 행정서비스 체제를 구축하는데 있어 자료로 활용함으로써 횡성군이 제공하는 민원행정서비스의 질적 향상에 기여하고자 한다. 
▷ 친절하고 쾌적한 민원실
▷ 고

In [ ]:
import os
import json
import glob
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings

from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore

# =========================
# 1. 모델 로드
# =========================
model_name = 'jhgan/ko-sbert-sts'
sbert_model = SentenceTransformer(model_name)
embedding_model = HuggingFaceEmbeddings(model_name=model_name)

# =========================
# 2. JSON 문서 로드
# =========================
folder_path = "./new2"
json_files = glob.glob(os.path.join(folder_path, "*.json"))

raw_documents = []
for file in json_files:
    with open(file, "r", encoding="utf-8-sig") as f:
        data = json.load(f)
        if "data" in data and isinstance(data["data"], list):
            for item in data["data"]:
                if "corpus" in item:
                    raw_documents.append(item["corpus"])

print(f"✅ 총 {len(raw_documents)}개의 JSON 문서를 로드했습니다.")

# =========================
# 3. 청킹 함수
# =========================
def chunk_documents(texts, chunk_size, chunk_overlap):
    splitter = CharacterTextSplitter(
        separator="",  # 문자 단위 청킹
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    full_text = "\n".join(texts)
    chunks = splitter.split_text(full_text)

    documents = [
        Document(page_content=chunk, metadata={"doc_id": f"doc_{i}"})
        for i, chunk in enumerate(chunks)
    ]
    return documents

# =========================
# 4. FAISS 인덱스 생성
# =========================
def create_langchain_faiss(documents, embedding_model):
    return FAISS.from_documents(documents, embedding=embedding_model)

# =========================
# 5. MultiVectorRetriever 실행 함수 (유사도 점수 포함)
# =========================
def run_multivector_retrieval(chunk_size, chunk_overlap, query, texts):
    print(f"\n===============================")
    print(f"🔍 MultiVectorRetriever: 청킹 {chunk_size}+{chunk_overlap}")
    print(f"===============================")

    documents = chunk_documents(texts, chunk_size, chunk_overlap)
    if not documents:
        print("❌ 청킹된 문서가 없습니다.")
        return

    # 1) FAISS 벡터스토어 생성
    vectorstore = create_langchain_faiss(documents, embedding_model)

    # 2) InMemory 문서 저장소
    docstore = InMemoryStore()
    for doc in documents:
        doc_id = doc.metadata["doc_id"]
        docstore.mset([(doc_id, doc)])

    # 3) MultiVectorRetriever 구성
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=docstore,
        id_key="doc_id"
    )

    # -- A. MultiVectorRetriever로 상위 문서 가져오기 (점수 없음)
    results = retriever.get_relevant_documents(query)

    # -- B. FAISS에서 직접 유사도 점수 가져오기
    #     (vectorstore.similarity_search_with_score)
    #     k=3으로 점수와 함께 문서 가져오기
    results_with_score = vectorstore.similarity_search_with_score(query, k=3)

    # 결과 출력
    if not results_with_score:
        print("❌ 검색 결과가 없습니다.")
        return


    print("\n🔎 [유사도 점수 직접 확인 - vectorstore 기준 (Top 3)]")
    for i, (doc_, score) in enumerate(results_with_score):
        print(f"\n▶️ [Top {i+1}] 유사도: {score:.4f}")
        print(doc_.page_content[:300], "...\n")

# =========================
# 6. 실행
# =========================
query = "횡성군 종합민원실의 민원행정서비스 조사는 며칠을 선정하여 실시하였어?"
chunking_configs = [
    (500, 50)
]

for chunk_size, chunk_overlap in chunking_configs:
    run_multivector_retrieval(chunk_size, chunk_overlap, query, raw_documents)


✅ 총 10개의 JSON 문서를 로드했습니다.

🔍 MultiVectorRetriever: 청킹 500+50
